In [1]:
import os
import re
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer


class SemanticChunker:
    def __init__(
        self, model_name="all-mpnet-base-v2", buffer_size=1, breakpoint_percentile=95
    ):
        """
        Inicializa el chunker semántico con configuraciones específicas.

        Parameters:
            model_name (str): Nombre del modelo de embeddings para usar.
            buffer_size (int): Cantidad de oraciones adyacentes a incluir en los embeddings.
            breakpoint_percentile (float): Percentil para calcular el umbral de puntos de quiebre.
        """
        self.model_name = model_name
        self.buffer_size = buffer_size
        self.breakpoint_percentile = breakpoint_percentile
        self.dense_embedding_model = SentenceTransformer(self.model_name)
        self.combined_sentences = []
        self.embeddings = []
        self.distances = []
        self.chunks = []

    def load_document(self, file_path):
        """
        Carga un documento en formato PDF y lo divide en oraciones.
        """
        from langchain.document_loaders import PyPDFLoader

        if not os.path.exists(file_path):
            raise FileNotFoundError(f"El archivo {file_path} no existe.")

        loader = PyPDFLoader(file_path)
        documents = loader.load()

        all_sentences = []
        for doc in documents:
            sentences = re.split(r"(?<=[.?!])\s+", doc.page_content)
            all_sentences.extend(sentences)

        return all_sentences

    def combine_sentences(self, sentences):
        """
        Combina oraciones adyacentes según el tamaño del buffer configurado.
        """
        combined_sentences = []
        for i, sentence in enumerate(sentences):
            combined = " ".join(
                sentences[max(0, i - self.buffer_size) : i + self.buffer_size + 1]
            )
            combined_sentences.append(combined)
        return combined_sentences

    def calculate_embeddings(self, combined_sentences):
        """
        Calcula los embeddings para las oraciones combinadas.
        """
        embeddings = self.dense_embedding_model.encode(combined_sentences)
        return [
            {"sentence": combined_sentences[i], "embedding": embeddings[i]}
            for i in range(len(combined_sentences))
        ]

    def calculate_distances(self, sentences):
        """
        Calcula las distancias coseno entre oraciones secuenciales.
        """
        similarities = []
        for i in range(len(sentences) - 1):
            embedding_current = sentences[i]["embedding"]
            embedding_next = sentences[i + 1]["embedding"]
            similarity = cosine_similarity([embedding_current], [embedding_next])[0][0]
            similarities.append(1 - similarity)  # Convertimos a distancia
        return similarities

    def detect_breakpoints(self, distances):
        """
        Detecta los puntos de quiebre basados en un percentil.
        """
        breakpoint_threshold = np.percentile(distances, self.breakpoint_percentile)
        indices_above_thresh = [
            i for i, x in enumerate(distances) if x > breakpoint_threshold
        ]
        return indices_above_thresh, breakpoint_threshold

    def generate_chunks(self, combined_sentences, indices_above_thresh):
        """
        Genera los chunks basados en los puntos de quiebre.
        """
        chunks = []
        start_index = 0

        for index in indices_above_thresh:
            end_index = index
            group = combined_sentences[start_index : end_index + 1]
            combined_text = " ".join([d["sentence"] for d in group])
            chunks.append(combined_text)
            start_index = index + 1

        if start_index < len(combined_sentences):
            combined_text = " ".join(
                [d["sentence"] for d in combined_sentences[start_index:]]
            )
            chunks.append(combined_text)

        return chunks

    def process_document(self, file_path):
        """
        Procesa un documento completo, devolviendo los chunks generados.
        """
        all_sentences = self.load_document(file_path)
        print(f"all_sentences quantity {len(all_sentences)}")
        self.combined_sentences = self.combine_sentences(all_sentences)
        print(f"combined_sentences quantity {len(self.combined_sentences)}")
        self.embeddings = self.calculate_embeddings(self.combined_sentences)
        print(f"embeddings quantity {len(self.embeddings)}")
        self.distances = self.calculate_distances(self.embeddings)
        print(f"distances quantity {len(self.distances)}")
        indices_above_thresh, _ = self.detect_breakpoints(self.distances)
        print(f"indices_above_thresh quantity {len(indices_above_thresh)}")
        self.chunks = self.generate_chunks(self.embeddings, indices_above_thresh)
        print(f"chunks quantity {len(self.chunks)}")
        return self.chunks

    def find_similar_chunk(self, query):
        """
        Encuentra el chunk más similar a una consulta.
        """
        query_embedding = self.dense_embedding_model.encode([query])[0]
        chunk_embeddings = [chunk["embedding"] for chunk in self.embeddings]
        similarities = cosine_similarity([query_embedding], chunk_embeddings)[0]
        most_similar_index = np.argmax(similarities)
        return {
            "query": query,
            "most_similar_chunk": self.embeddings[most_similar_index]["sentence"],
            "similarity_score": similarities[most_similar_index],
        }

/Users/marcofura/anaconda3/envs/actividad_1/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
# from chunking.semantic_chunker import SemanticChunker

# Inicializa el chunker
chunker = SemanticChunker(buffer_size=1, breakpoint_percentile=95)

# Procesa el documento
file_path = "./bucket/Oscar Wilde - El Retrato de Dorian Gray.pdf"
chunks = chunker.process_document(file_path)

print(f"Se han generado {len(chunks)} chunks.")
print("Primer chunk:", chunks[0])

all_sentences quantity 6680
combined_sentences quantity 6680
embeddings quantity 6680
distances quantity 6679
indices_above_thresh quantity 334
chunks quantity 335
Se han generado 335 chunks.
Primer chunk: 1 El Retrato de Dorian Gray
Oscar Wilde
textos.infobiblioteca digital abierta
2 1 El Retrato de Dorian Gray
Oscar Wilde
textos.infobiblioteca digital abierta
2 Texto núm. El Retrato de Dorian Gray
Oscar Wilde
textos.infobiblioteca digital abierta
2 Texto núm. 252
Título : El Retrato de Dorian Gray
Autor : Oscar Wilde
Etiquetas : Novela
Editor : Edu Robsy
Fecha de creación : 20 de mayo de 2016
Fecha de modificación : 3 de noviembre de 2023
Edita textos.info
Maison Carrée
c/ des Ramal, 48
07730 Alayor - Menorca
Islas Baleares
España
Más textos disponibles en http://www.textos.info
3 Texto núm. 252
Título : El Retrato de Dorian Gray
Autor : Oscar Wilde
Etiquetas : Novela
Editor : Edu Robsy
Fecha de creación : 20 de mayo de 2016
Fecha de modificación : 3 de noviembre de 2023
Edita textos

In [3]:
query = "Cómo era físicamente Dorian Gray?"
result = chunker.find_similar_chunk(query)

print("Query:", result["query"])
print("Chunk más similar:", result["most_similar_chunk"])
print("Puntaje de similitud:", result["similarity_score"])

Query: Cómo era físicamente Dorian Gray?
Chunk más similar: Dorian Gray se sintió enfermar de miedo. —No sé de qué me habla — tartamudeó —. Nunca he oído ese nombre.
Puntaje de similitud: 0.85815144


In [5]:
chunks[2]

'—¿No lo vas a enviar a ningún sitio? ¿Por qué, mi querido amigo? ¿Qué \nrazón podrías aducir? ¿Por qué, mi querido amigo? ¿Qué \nrazón podrías aducir? ¿Por qué sois unas gentes tan raras los pintores? ¿Qué \nrazón podrías aducir? ¿Por qué sois unas gentes tan raras los pintores? Hacéis cualquier cosa para ganaros una reputación, pero, tan pronto como \nla tenéis, se diría que os sobra. ¿Por qué sois unas gentes tan raras los pintores? Hacéis cualquier cosa para ganaros una reputación, pero, tan pronto como \nla tenéis, se diría que os sobra. Es una tontería, porque en el mundo sólo \nhay algo peor que ser la persona de la que se habla y es ser alguien de \nquien no se habla. Hacéis cualquier cosa para ganaros una reputación, pero, tan pronto como \nla tenéis, se diría que os sobra. Es una tontería, porque en el mundo sólo \nhay algo peor que ser la persona de la que se habla y es ser alguien de \nquien no se habla. Un retrato como ése te colocaría muy por encima de \ntodos los pintore